In [1]:
import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

scopes = [ "https://www.googleapis.com/auth/youtube",
"https://www.googleapis.com/auth/youtube.force-ssl",
"https://www.googleapis.com/auth/youtube.readonly",
"https://www.googleapis.com/auth/youtubepartner"]

In [2]:
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

api_service_name = "youtube"
api_version = "v3"
client_secrets_file = "../secret_key/client_secret.json"
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
    client_secrets_file, scopes)
credentials = flow.run_console()
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=699806263406-koq1jouhe37k38dvvshpu98ib9m5o53g.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutubepartner&state=5uBGCs1Y4suNrW4FjM9UQZMtYIHv0o&prompt=consent&access_type=offline
Enter the authorization code: 4/yQG65eicxJDYjZPwA17CwFBtV_TLk8eO6BtmOXP9zonRx5KqY_ySW40


In [4]:
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)

In [37]:
my_subscriptions = []

request = youtube.subscriptions().list(
    part='snippet',
    mine='true',
    maxResults='50',
    pageToken=''
)
response = request.execute()

print(response)

{'kind': 'youtube#subscriptionListResponse', 'etag': '"xwzn9fn_LczrfK9QS3iZcGzqRGs/02WT683tbze2mimv12pN3ue32j4"', 'nextPageToken': 'CDIQAA', 'pageInfo': {'totalResults': 183, 'resultsPerPage': 50}, 'items': [{'kind': 'youtube#subscription', 'etag': '"xwzn9fn_LczrfK9QS3iZcGzqRGs/rdPrNR38Dp1aQH96RDBpOvMBhmk"', 'id': 'XMm1RFsYKmvBkbp-hf4Ryg0vUu7KxIYHgJDa45uTiuA', 'snippet': {'publishedAt': '2020-04-04T13:33:37.056Z', 'title': '승우아빠 일상채널', 'description': '🎁 승소리 포장 출력소 🎁\n🗯️ 게임, 일상, 흑우리뷰 🗯️', 'resourceId': {'kind': 'youtube#channel', 'channelId': 'UCs1R9zP6rH2GlTNHUI5SBkw'}, 'channelId': 'UCWTLxUY-0lkcLHFYz3swO1Q', 'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/-6XLk13REPnc/AAAAAAAAAAI/AAAAAAAAAAA/s8UW3SR2K8o/s88-c-k-no-mo-rj-c0xffffff/photo.jpg'}, 'medium': {'url': 'https://yt3.ggpht.com/-6XLk13REPnc/AAAAAAAAAAI/AAAAAAAAAAA/s8UW3SR2K8o/s240-c-k-no-mo-rj-c0xffffff/photo.jpg'}, 'high': {'url': 'https://yt3.ggpht.com/-6XLk13REPnc/AAAAAAAAAAI/AAAAAAAAAAA/s8UW3SR2K8o/s800-c-k-no-mo-rj

In [43]:
def get_subscriptions():
    my_subscriptions = []
    next_page_token = ''
    
    while(True):
        print('here')
        request = youtube.subscriptions().list(
            part='snippet',
            mine='true',
            maxResults='50',
            pageToken=next_page_token
        )
        response = request.execute()
        my_subscriptions += response['items']
        
        if 'nextPageToken' not in response.keys():
            break
        next_page_token = response['nextPageToken']
    return my_subscriptions

In [44]:
my_subscriptions = get_subscriptions()

here
here
here
here


In [46]:
len(my_subscriptions)

180

In [57]:
my_subscriptions[0]

{'kind': 'youtube#subscription',
 'etag': '"xwzn9fn_LczrfK9QS3iZcGzqRGs/rdPrNR38Dp1aQH96RDBpOvMBhmk"',
 'id': 'XMm1RFsYKmvBkbp-hf4Ryg0vUu7KxIYHgJDa45uTiuA',
 'snippet': {'publishedAt': '2020-04-04T13:33:37.056Z',
  'title': '승우아빠 일상채널',
  'description': '🎁 승소리 포장 출력소 🎁\n🗯️ 게임, 일상, 흑우리뷰 🗯️',
  'resourceId': {'kind': 'youtube#channel',
   'channelId': 'UCs1R9zP6rH2GlTNHUI5SBkw'},
  'channelId': 'UCWTLxUY-0lkcLHFYz3swO1Q',
  'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/-6XLk13REPnc/AAAAAAAAAAI/AAAAAAAAAAA/s8UW3SR2K8o/s88-c-k-no-mo-rj-c0xffffff/photo.jpg'},
   'medium': {'url': 'https://yt3.ggpht.com/-6XLk13REPnc/AAAAAAAAAAI/AAAAAAAAAAA/s8UW3SR2K8o/s240-c-k-no-mo-rj-c0xffffff/photo.jpg'},
   'high': {'url': 'https://yt3.ggpht.com/-6XLk13REPnc/AAAAAAAAAAI/AAAAAAAAAAA/s8UW3SR2K8o/s800-c-k-no-mo-rj-c0xffffff/photo.jpg'}}}}

In [49]:
import pandas as pd

In [55]:
df_my_sub = pd.DataFrame(my_subscriptions)

In [59]:
df_my_sub.to_csv('my_subscriptions.csv')

In [61]:
pd.read_csv('my_subscriptions.csv', index_col = 0)

,etag,id,kind,snippet
0,"""xwzn9fn_LczrfK9QS3iZcGzqRGs/rdPrNR38Dp1aQH96R...",XMm1RFsYKmvBkbp-hf4Ryg0vUu7KxIYHgJDa45uTiuA,youtube#subscription,"{'publishedAt': '2020-04-04T13:33:37.056Z', 't..."
1,"""xwzn9fn_LczrfK9QS3iZcGzqRGs/dhNNLrIxUTIz2xok-...",XMm1RFsYKmtAqoa2erMGw1z5bX_IrrDeu4d_EgVAFFc,youtube#subscription,"{'publishedAt': '2020-03-05T14:41:04.125Z', 't..."
2,"""xwzn9fn_LczrfK9QS3iZcGzqRGs/BIvGGGHh-T66W_vMC...",XMm1RFsYKmtAqoa2erMGw2-_zhRy5SidEeP32uxncVg,youtube#subscription,"{'publishedAt': '2020-01-15T09:40:40.624Z', 't..."
3,"""xwzn9fn_LczrfK9QS3iZcGzqRGs/sanQt84D22H03bNcr...",XMm1RFsYKmvBkbp-hf4RyqUr5ndiZnl5ukfs2jtOQco,youtube#subscription,"{'publishedAt': '2019-07-26T10:18:11.240Z', 't..."
4,"""xwzn9fn_LczrfK9QS3iZcGzqRGs/G0Lm9cRJKn36wlx7o...",XMm1RFsYKmtAqoa2erMGwz4bh0-FoPArY0TrvLeeQVY,youtube#subscription,"{'publishedAt': '2019-06-04T15:26:43.980Z', 't..."
5,"""xwzn9fn_LczrfK9QS3iZcGzqRGs/ff50cG1yssS-McjuJ...",XMm1RFsYKmsKG2GYc6kuBpZuKjG2xWAgDZWcWW9bkSA,youtube#subscription,"{'publishedAt': '2018-11-04T15:02:36.322Z', 't..."
6,"""xwzn9fn_LczrfK9QS3iZcGzqRGs/KoO0yQA0iQPthVoTC...",XMm1RFsYKmvBkbp-hf4RygOTdw4KmMtaKeKXvX45JE0,youtube#subscription,"{'publishedAt': '2020-02-29T02:59:19.022Z', 't..."
7,"""xwzn9fn_LczrfK9QS3iZcGzqRGs/87rfDvTrar2H01iUz...",XMm1RFsYKmsKG2GYc6kuBlw1R0mEvtibvr_bOSe2GOY,youtube#subscription,"{'publishedAt': '2020-03-24T23:40:32.641Z', 't..."
8,"""xwzn9fn_LczrfK9QS3iZcGzqRGs/lhzZkEEd33CqFnIwg...",XMm1RFsYKmtAqoa2erMGw6K2NEyuW_VyHWRhvtKWzwI,youtube#subscription,"{'publishedAt': '2019-10-26T09:12:36.219Z', 't..."
9,"""xwzn9fn_LczrfK9QS3iZcGzqRGs/_cJc-rq-rBUd_jIw8...",XMm1RFsYKmtAqoa2erMGwxdquHhweIV3Xv-t3zVtNLw,youtube#subscription,"{'publishedAt': '2019-01-01T15:53:21.972Z', 't..."
